In [1]:
import psycopg2
from psycopg2  import extras
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

DATABASE = 'postgresql'
USER = 'root'
PASSWORD = 'root'
HOST = 'postgres' # shared-networkに所属しているpostgres containerを指定できる
PORT = '5432'
DB_NAME = 'sandbox'

CONNECT_STR = '{}://{}:{}@{}:{}/{}'.format(DATABASE, USER, PASSWORD, HOST, PORT, DB_NAME)
engine = create_engine(CONNECT_STR)

conn = engine.connect()
res = conn.execute("select datname, datdba, encoding, datcollate, datctype from pg_database;")

for row in res:
    print(row)

('postgres', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('sandbox', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template1', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template0', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')


In [2]:
# create table
tablename = 'x005_student_club'

q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename)
res = conn.execute(q1)

q2 = """
CREATE TABLE  {} (
    std_id integer NOT NULL,
    club_id integer NOT NULL,
    club_name varchar(256) NOT NULL,
    main_club_flg varchar(256) NOT NULL
);
""".format(tablename)

if sum(1 for _ in res) == 0:
    print("create")
    conn.execute(q2)
    res = conn.execute(q1)
    sum(1 for _ in res)

In [3]:
# insert data
data = (
    (100, 1, '野球', 'Y'),
    (100, 2, '吹奏楽', 'N'),
    (200, 2, '吹奏楽', 'N'),
    (200, 3, 'バドミントン', 'Y'),
    (200, 4, 'サッカー', 'N'),
    (300, 4, 'サッカー', 'N'),
    (400, 5, '水泳', 'N'),
    (500, 6, '囲碁', 'N'),
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
            Column('std_id', Integer, primary_key=False),
            Column('club_id', Integer, primary_key=False),
            Column('club_name', String, primary_key=False),
            Column('main_club_flg', String),
           )

for i in data:
    ins = tbl.insert().values(std_id=i[0], club_id=i[1], club_name=i[2], main_club_flg=i[3])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

(100, 1, '野球', 'Y')
(100, 2, '吹奏楽', 'N')
(200, 2, '吹奏楽', 'N')
(200, 3, 'バドミントン', 'Y')
(200, 4, 'サッカー', 'N')
(300, 4, 'サッカー', 'N')
(400, 5, '水泳', 'N')
(500, 6, '囲碁', 'N')


In [4]:
q = f"""
select std_id,
    case when count(*) = 1
        then max(club_id)
    else max(
        case when main_club_flg = 'Y'
            then club_id
        else null end
    ) end as main_club
from {tablename}
group by std_id;
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select std_id,
    case when count(*) = 1
        then max(club_id)
    else max(
        case when main_club_flg = 'Y'
            then club_id
        else null end
    ) end as main_club
from x005_student_club
group by std_id;

(300, 4)
(500, 6)
(100, 1)
(200, 3)
(400, 5)
